In [3]:
import re
import qiskit
import json
import time
import os
import numpy as np
from qiskit import transpile, QuantumCircuit
import time

from fast_generator import fc_tree_commute_lookahead_fast
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth
# from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
import rustiq

from qiskit.transpiler import CouplingMap

print(qiskit.__version__)

1.1.1


In [4]:
def convert_to_circuit(circuit: list) -> QuantumCircuit:
    """
    Converts a circuit into a CNOT + H + S + RZ(theta) to a quantumcircuit
    """
    num_qubits=len(set([num for op in circuit for num in op[1]]))
    new_circuit=QuantumCircuit(num_qubits)
    for gate in circuit:
        op=gate[0]
        params=gate[1]
        assert op in ["CNOT", "CZ", "H", "S", "Sd", "X", "Z", "SqrtX", "SqrtXd"], f"op not allowed {op}"
        if op=="CNOT":
            new_circuit.cx(params[0], params[1])
        elif op=="CZ":
            new_circuit.cz(params[0], params[1])
        elif op=="H":
            new_circuit.h(params[0])
        elif op=="S":
            new_circuit.s(params[0])
        elif op=="Sd":
            new_circuit.sdg(params[0])
        elif op=="X":
            new_circuit.x(params[0])
        elif op=="Z":
            new_circuit.z(params[0])
        elif op=="SqrtX":
            new_circuit.sx(params[0])
        elif op=="SqrtXd":
            new_circuit.sxdg(params[0])
    return new_circuit

In [5]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            combined_time = end_time - start_time
            
            # Measure time for Qiskit method
            start_time = time.time()
            origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            qiskit_time = end_time - start_time

            # Measure time for RustiQ method
            start_time = time.time()
            circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            qiskit_circ_rustiq=convert_to_circuit(circuit)
            # print(qiskit_circ_rustiq)
            # can't optimize with level 3 since rustiq returns an optimized circuit missing some single qubit gates.
            qiskit_circ_rustiq = transpile(qiskit_circ_rustiq, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    "qiskit_time": qiskit_time,
                    "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    "rustiq_method": qiskit_circ_rustiq.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    "rustiq_method": qiskit_circ_rustiq.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_grid/test_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology_ibm_grid/test_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [6]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, lookahead_size = 5, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    qiskit_circ_rustiq=convert_to_circuit(circuit)
    # print(qiskit_circ_rustiq)
    # can't optimize with level 3 since rustiq returns an optimized circuit missing some single qubit gates.
    qiskit_circ_rustiq = transpile(qiskit_circ_rustiq, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": qiskit_circ_rustiq.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            "rustiq_method": qiskit_circ_rustiq.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_grid/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology_ibm_grid/test_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [7]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,] #12]
orbitals_list = [4, 6, 8, 12, 16, 20,] #24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, lookahead_size = 5, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.519160270690918, 'combined_time': 0.2732691764831543, 'qiskit_time': 0.27725958824157715, 'rustiq_time': 0.160081148147583}, 'gate_counts': {'our_method': 66, 'combined_method': 41, 'qiskit_method': 41, 'rustiq_method': 84}, 'circuit_entangling_depth': {'our_method': 56, 'combined_method': 34, 'qiskit_method': 41, 'rustiq_method': 72}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 1.5495874881744385, 'combined_time': 0.8403606414794922, 'qiskit_time': 0.6208512783050537, 'rustiq_time': 0.7512326240539551}, 'gate_counts': {'our_method': 436, 'combined_method': 165, 'qiskit_method': 183, 'rustiq_method': 632}, 'circuit_entangling_depth': {'our_method': 317, 'combined_method': 136, 'qiskit_method': 180, 'rustiq_method': 434}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 6.152129173

In [8]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, lookahead_size = 10)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 32.605815410614014, 'combined_time': 28.063989400863647, 'qiskit_time': 9.4660165309906, 'rustiq_time': 22.092109441757202}, 'gate_counts': {'our_method': 18712, 'combined_method': 9933, 'qiskit_method': 9123, 'rustiq_method': 17211}, 'circuit_entangling_depth': {'our_method': 9367, 'combined_method': 6287, 'qiskit_method': 8041, 'rustiq_method': 9106}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 3.547217607498169, 'combined_time': 2.7195138931274414, 'qiskit_time': 1.3320658206939697, 'rustiq_time': 1.767524003982544}, 'gate_counts': {'our_method': 1716, 'combined_method': 650, 'qiskit_method': 988, 'rustiq_method': 1848}, 'circuit_entangling_depth': {'our_method': 996, 'combined_method': 473, 'qiskit_method': 894, 'rustiq_method': 1109}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_H2O.json'}
LiH.json
{'num_pa

In [9]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, lookahead_size = 10)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.4507935047149658, 'combined_time': 0.43914008140563965, 'qiskit_time': 0.05584979057312012, 'rustiq_time': 0.09578919410705566}, 'gate_counts': {'our_method': 237, 'combined_method': 60, 'qiskit_method': 28, 'rustiq_method': 231}, 'circuit_entangling_depth': {'our_method': 117, 'combined_method': 46, 'qiskit_method': 18, 'rustiq_method': 102}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 1.4040133953094482, 'combined_time': 1.5492284297943115, 'qiskit_time': 0.14502763748168945, 'rustiq_time': 0.21342968940734863}, 'gate_counts': {'our_method': 683, 'combined_method': 199, 'qiskit_method': 83, 'rustiq_method': 515}, 'circuit_entangling_depth': {'our_method': 336, 'combined_method': 130, 'qiskit_method': 42, 'rustiq_method': 243}, 'test_paulis_fi

In [10]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, lookahead_size = 10)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 4.3212361335754395, 'combined_time': 3.7086567878723145, 'qiskit_time': 0.19167232513427734, 'rustiq_time': 0.3820643424987793}, 'gate_counts': {'our_method': 770, 'combined_method': 257, 'qiskit_method': 542, 'rustiq_method': 815}, 'circuit_entangling_depth': {'our_method': 396, 'combined_method': 174, 'qiskit_method': 346, 'rustiq_method': 406}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 12.695252895355225, 'combined_time': 11.308904647827148, 'qiskit_time': 0.9813215732574463, 'rustiq_time': 0.8672194480895996}, 'gate_counts': {'our_method': 2267, 'combined_method': 792, 'qiskit_method': 1670, 'rustiq_method': 2278}, 'circuit_entangling_depth': {'our_method': 1191, 'combined_method': 592, 'qiskit_method': 1131, 'rustiq_method': 1167}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_